# Анализ текста

## Морфологический анализ

Морфологический анализ текста - это процесс изучения формы или структуры слов.

Когда мы говорим о форме слова, мы имеем в виду его части, такие как корень, суффикс, приставка и окончание. 

Рассмотрим пример:

В слове "бегать" есть корень "бег-", к которому добавляется суффикс "-ать". Это делает слово "бегать" глаголом, означающим действие бега. Также, если мы добавим приставку "по-" и сделаем "побегать", это будет означать "бегать немного".

Теперь рассмотрим окончания. Например, если мы возьмем слово "бегает", то его окончание "-ет" указывает на то, что это глагол, который описывает действие, выполняемое кем-то или чем-то.

Морфологический анализ текста помогает нам понять эти части слова и их значения. Он помогает нам понять, как слова изменяются в разных формах, чтобы передать разные идеи или оттенки значения.

Таким образом, морфологический анализ текста помогает нам разбираться в строении слов и их формах, чтобы мы могли лучше понять, что именно хотят нам сказать.

### Подготовка библиотек

In [1]:
import os
from IPython.display import clear_output

dir_import = 'import'
os.makedirs(dir_import, exist_ok=True)

In [2]:
!pip install pymorphy2
!pip install pymorphy3
!pip install -U pymorphy2-dicts-ru
clear_output()

In [3]:
import pymorphy2
import re
import string
import csv

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksioprime/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Ввод текста

In [4]:
text = """
    Python - это простой в использовании, но мощный в своих возможностях язык программирования. 
    Он широко используется для веб-разработки, научных вычислений, искусственного интеллекта и многого другого.
    """

### Обработка текста

In [5]:
def text_processing(text):
    translator = str.maketrans('', '', string.punctuation)
    stop_set = set(stopwords.words('russian'))
    mystem = Mystem()
    
    words = text.lower().translate(translator).split()
    tokens = [word for word in words if word not in stop_set]
    # tokens_lemma = list(filter(lambda x: x not in [' ', '\n'], mystem.lemmatize(' '.join(tokens))))
    return tokens

tokens_ready = text_processing(text)

### Вывод граммем (меток)

In [6]:
morph = pymorphy2.MorphAnalyzer(lang='ru')
for word in tokens_ready:
    parsed_word = morph.parse(word)[0]
    tags = str(parsed_word.tag).split(',')
    print(f"{word}: {tags}")

python: ['LATN']
это: ['PRCL']
простой: ['ADJF', 'Qual femn', 'sing', 'gent']
использовании: ['NOUN', 'inan', 'neut sing', 'loct']
мощный: ['ADJF', 'Qual masc', 'sing', 'nomn']
своих: ['ADJF', 'Apro', 'Anph plur', 'gent']
возможностях: ['NOUN', 'inan', 'femn plur', 'loct']
язык: ['NOUN', 'inan', 'masc sing', 'accs']
программирования: ['NOUN', 'inan', 'neut sing', 'gent']
широко: ['ADVB']
используется: ['VERB', 'impf', 'intr sing', '3per', 'pres', 'indc']
вебразработки: ['NOUN', 'inan', 'femn sing', 'gent']
научных: ['ADJF', 'Qual plur', 'gent']
вычислений: ['NOUN', 'inan', 'neut plur', 'gent']
искусственного: ['ADJF', 'Qual neut', 'sing', 'gent']
интеллекта: ['NOUN', 'inan', 'masc sing', 'gent']
многого: ['NPRO', 'neut sing', 'gent']
другого: ['ADJF', 'Subx', 'Apro neut', 'sing', 'gent']


### Вывод граммем (расшифровка)
https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#russian-pos

In [7]:
# Функция для загрузки словаря грамем из CSV файла
def load_grammemes_dict(csv_file):
    grammemes_dict = {}
    with open(csv_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            internal_id = row['internal_id']
            description = row['description']
            grammemes_dict[internal_id] = description
    return grammemes_dict
    
# Загрузка словаря грамем из CSV файла
import_grammemes = os.path.join(dir_import, "grammemes.csv")
grammemes_dict = load_grammemes_dict(import_grammemes)

In [8]:
for word in tokens_ready:
    parsed_word = morph.parse(word)[0]
    grammemes = parsed_word.tag.grammemes
    grammemes_rus = [ grammemes_dict.get(grammeme, grammeme) for grammeme in grammemes ]        
    print(f"{word}: {grammemes_rus}")

python: ['LATN']
это: ['частица']
простой: ['качественное', 'имя прилагательное (полное)', 'единственное число', 'женский род', 'родительный падеж']
использовании: ['имя существительное', 'неодушевлённое', 'средний род', 'единственное число', 'предложный падеж']
мощный: ['качественное', 'мужской род', 'имя прилагательное (полное)', 'единственное число', 'именительный падеж']
своих: ['местоименное', 'имя прилагательное (полное)', 'родительный падеж', 'Анафорическое (местоимение)', 'множественное число']
возможностях: ['имя существительное', 'неодушевлённое', 'женский род', 'предложный падеж', 'множественное число']
язык: ['имя существительное', 'мужской род', 'неодушевлённое', 'винительный падеж', 'единственное число']
программирования: ['имя существительное', 'неодушевлённое', 'средний род', 'единственное число', 'родительный падеж']
широко: ['наречие']
используется: ['несовершенный вид', 'глагол (личная форма)', 'единственное число', 'непереходный', '3 лицо', 'изъявительное наклонение

### Работа с граммемами

- Часть речи можно получить через атрибут POS (```parsed_word.tag.POS```)
- Падеж выделяется у существительных, полных прилагательных, полных причастий, числительных и местоимений. Получить его можно через атрибут case (```parsed_word.tag.case```)
- Число можно получить через атрибут number (```parsed_word.tag.number```)
- Род можно пролучить через атрибут gender (```parsed_word.tag.gender```)

### Получение имён существительных и определение из падежа

In [9]:
for word in tokens_ready:
    parsed_word = morph.parse(word)[0]
    if parsed_word.tag.POS == 'NOUN':
        case = parsed_word.tag.case
        print(f"{word}: {grammemes_dict.get(case, case)}")

использовании: предложный падеж
возможностях: предложный падеж
язык: винительный падеж
программирования: родительный падеж
вебразработки: родительный падеж
вычислений: родительный падеж
интеллекта: родительный падеж


### Склонение слова

In [10]:
word = 'кошка'
my_word = morph.parse(word)[0]

for word in my_word.lexeme:
    case = word.tag.case
    case_rus = grammemes_dict.get(case, case)
    number = word.tag.number
    number_rus = grammemes_dict.get(number, number)
    print(f"{word.word}: {case}, {number} ({case_rus}, {number_rus})")

кошка: nomn, sing (именительный падеж, единственное число)
кошки: gent, sing (родительный падеж, единственное число)
кошке: datv, sing (дательный падеж, единственное число)
кошку: accs, sing (винительный падеж, единственное число)
кошкой: ablt, sing (творительный падеж, единственное число)
кошкою: ablt, sing (творительный падеж, единственное число)
кошке: loct, sing (предложный падеж, единственное число)
кошки: nomn, plur (именительный падеж, множественное число)
кошек: gent, plur (родительный падеж, множественное число)
кошкам: datv, plur (дательный падеж, множественное число)
кошек: accs, plur (винительный падеж, множественное число)
кошками: ablt, plur (творительный падеж, множественное число)
кошках: loct, plur (предложный падеж, множественное число)


### Склонение слова с числительным

In [11]:
word = 'кошка'
my_word = morph.parse(word)[0]

for i in range(8):
    word_i = my_word.make_agree_with_number(i).word
    print(f"{i} {word_i}")

0 кошек
1 кошка
2 кошки
3 кошки
4 кошки
5 кошек
6 кошек
7 кошек


## Синтаксический анализ текста

Синтаксический анализ текста - это процесс, в ходе которого мы изучаем, как слова в предложении связаны между собой.

Когда мы говорим или пишем, мы используем слова, чтобы передать свои мысли и идеи. Эти слова не просто располагаются одно за другим, они также имеют определенные отношения между собой. Например, в предложении "Кот ловит мышь", слово "кот" выполняет действие, а слово "мышь" является объектом этого действия.

Синтаксический анализ текста помогает нам понять эти отношения между словами. Он помогает нам определить, какие слова являются частью предложения, какие слова выполняют действие, какие слова описывают субъект или объект, и как эти слова связаны друг с другом.

В результате синтаксического анализа мы можем построить дерево, которое показывает, как каждое слово в предложении связано с другими словами. Это дерево называется деревом зависимостей. Оно помогает нам лучше понять структуру предложения и какие идеи или действия оно передает.

Примеры ролей в дереве:

- **nsubj**: Субъект (Subject) - подлежащее предложения. Это слово или фраза, которая выполняет действие, описанное глаголом. Например, в предложении "Кот ловит мышь", "кот" является подлежащим.

- **root**: Корень (Root) - это корневой токен зависимостей. Он не имеет родителя и является центральным токеном зависимостей в дереве. Обычно это глагол или главный глагольный комплекс в предложении.

- **advmod**: Наречие (Adverbial Modifier) - это наречие, которое модифицирует глагол, прилагательное, другое наречие или даже целое предложение. Оно обычно указывает на время, место, причину, степень или образ выполнения действия. Например, в предложении "Он очень быстро бегает", "очень" - наречие, модифицирующее глагол "бегает".

- **obl**: Объект (Object) - это комплемент, который связан с глаголом и обозначает объект действия. Например, в предложении "Она купила новый автомобиль", "новый автомобиль" - объект.

- **dobj**: Прямой объект (Direct Object) - подкатегория объекта, который является прямым объектом действия.

- **attr**: Атрибут (Attribute) - это слово, которое указывает на свойство или характеристику субъекта.

- **subj**: Подлежащее (Subject) - альтернативное обозначение для подлежащего.

- **aux**: Вспомогательный глагол (Auxiliary) - это вспомогательный глагол, который используется для образования глагольной формы, но сам не несет основного смысла.

- **conj**: Сочинение (Conjunct) - это токен, который является частью координации, например, в списке или параллельном выражении.

### Установка библиотек

In [12]:
!pip install numpy
!pip install spacy
clear_output()

### Импорт библиотек

In [19]:
# Импортируем библиотеку spaCy для обработки естественного языка
import spacy
# Импортируем модуль displacy из spaCy для визуализации зависимостей
from spacy import displacy
# Импортируем русский языковой модельный пакет большого размера
import importlib
try:
    importlib.import_module('ru_core_news_lg')
    print(f"Модель установлена")
except ImportError:
    print(f"Модель не найдена. Загружаем...")
    !python -m spacy download ru_core_news_lg
    clear_output()
finally:
    import ru_core_news_lg

Модель установлена


### Ввод текста

In [20]:
text = """
Я пришел домой поздно вечером.
"""

### Построение графика синтаксических зависимостей

In [21]:
# Загружаем русскую языковую модель большого размера
en_nlp = ru_core_news_lg.load()
# Применяем модель к тексту, создавая объект Doc
doc = en_nlp(text)
# Визуализируем синтаксические зависимости в тексте
displacy.render(doc, style="dep")

## Семантический анализ текста

Семантический анализ текста - это способ понимания значения слов и их связей в предложении или тексте. 

Когда мы читаем или слушаем текст, мы не просто видим отдельные слова, но и стараемся понять, о чем идет речь, какие мысли автор хочет передать. Семантический анализ помогает нам в этом.

Для понимания смысла текста нам нужно знать значения слов и как они сочетаются друг с другом. Например, если мы видим слова "кот" и "мышь", мы понимаем, что кот может поймать мышь, потому что это типичное поведение котов. Это понимание значений слов и их связей - это и есть семантический анализ.

Семантический анализ также помогает нам понять отношения между предложениями или частями текста. Например, если в одном предложении говорится "Кот поймал мышь", а в другом "Мышь попыталась убежать", мы понимаем, что первое предложение описывает действие, которое произошло до второго.

Таким образом, семантический анализ текста помогает нам не просто читать слова, а понимать, о чем речь и какие мысли или идеи автор хочет передать.

### Установка библиотек

In [22]:
!pip install textblob
!pip install spacy
!pip install dostoevsky
!python3 -m dostoevsky download fasttext-social-network-model
clear_output()

### Импорт библиотек

In [24]:
import spacy
from nltk import Tree
from spacy import displacy
try:
    importlib.import_module('ru_core_news_lg')
    print(f"Модель установлена")
except ImportError:
    print(f"Модель не найдена. Загружаем...")
    !python -m spacy download ru_core_news_lg
    clear_output()
finally:
    import ru_core_news_lg

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

Модель установлена


### Ввод текста

In [ ]:
text = """
Сидоров, ты тупой!
"""

### Извлечение именованных сущностей

In [ ]:
en_nlp = ru_core_news_lg.load()
doc = en_nlp(text)
for named_entity in doc.ents:
    label = named_entity.label_
    explain_label = spacy.explain(label)
    print(f"{named_entity}: {label} ({explain_label})")

### Анализ тональности
https://github.com/bureaucratic-labs/dostoevsky/tree/master

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

results = model.predict([text], k=5)[0]
print("\n".join([f"{key}: {value}" for key, value in results.items()]))